In [ ]:
!pip install -q llama-index-embeddings-huggingface
!pip install -q torch
!pip install -q sentence-transformers
!pip install -q llama_index
!pip install -q tensorflow-io
!pip install -q llama-index-vector-stores-chroma
!pip install -q transformers
!pip install -q elasticsearch
!pip install -q peft
!pip install -q langchain
!pip install -q lancedb
!pip install -q unstructured
!pip install -U -q langchain-community
!pip install -q llama-index-llms-huggingface

In [ ]:
# Импорт необходимых библиотек и модулей
from sentence_transformers import SentenceTransformer
from llama_index.core import Document, Settings, SimpleDirectoryReader, StorageContext, VectorStoreIndex
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import JSONLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import ElasticsearchStore
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.document_transformers import LongContextReorder
from langchain.prompts import PromptTemplate
from typing import Any, Generator
from peft import AutoPeftModelForCausalLM, PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, AutoModel
import torch
import torch.nn.functional as F
from llama_index.core.llms import CustomLLM, CompletionResponse, CompletionResponseGen, LLMMetadata
from llama_index.core.llms.callbacks import llm_completion_callback
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core import PromptTemplate, get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
# Импорт библиотек TensorFlow и TensorFlow I/O
import tensorflow as tf
import tensorflow_io as tfio

# Импорт дополнительных модулей из TensorFlow
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
# Установка Elasticsearch
%%capture
!pip install elasticsearch==8.8.0

In [ ]:
%%bash

rm -rf elasticsearch*
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.8.0-linux-x86_64.tar.gz
tar -xzf elasticsearch-8.8.0-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-8.8.0/
umount /sys/fs/cgroup
apt install cgroup-tools

In [ ]:
try:
    import os
    import elasticsearch
    from elasticsearch import Elasticsearch
    import numpy as np
    import pandas as pd
    import sys
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    from elasticsearch import helpers

except Exception as e:
    print(f"error: {e}")

In [ ]:
# Запуск Elasticsearch в фоновом режиме
%%bash --bg

sudo -H -u daemon elasticsearch-8.8.0/bin/elasticsearch

In [ ]:
 # Импорт библиотеки для работы со временем
import time
time.sleep(120)

In [ ]:
# Проверка работающих процессов Elasticsearch
!ps -ef | grep elastic

In [ ]:
# Настройка паролей для Elasticsearch
!/content/elasticsearch-8.8.0/bin/elasticsearch-setup-passwords auto -url "https://localhost:9200"

In [ ]:
# Проверка доступности сервера Elasticsearch
!curl --cacert /content/elasticsearch-8.8.0/config/certs/http_ca.crt -u elastic -H 'Content-Type: application/json' -XGET https://localhost:9200/?pretty=true

In [ ]:
# Установка учетных данных для подключения к Elasticsearch
username = 'elastic'

password = 'JstXEztaqQEigZ8TOTxl'

# Создание подключения к Elasticsearch
es = Elasticsearch(['https://localhost:9200'], basic_auth=(username, password), ca_certs="/content/elasticsearch-8.8.0/config/certs/http_ca.crt")

resp = dict(es.info())

resp

In [ ]:
# Проверка работоспособности сервера
es.ping()

In [ ]:
# Путь к JSON-файлу
json_file_path = '/content/ConfluencePages...json'

# Загрузка данных из JSON-файла
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Создание списка документов с использованием LlamaIndex
documents = [Document(
    text=page['text'],
    metadata={"title": page['title'], "link": page['link'], "date": page['date'], "author": page['author']},
) for page in data['pages']]

# Вывод текста первого документа
print((documents[0].text))

documents[0]

In [ ]:
# Разбиение документов на узлы с использованием SentenceSplitter
parser = SentenceSplitter(chunk_size=400, chunk_overlap=50)
# Получение узлов из документов
nodes = parser.get_nodes_from_documents(documents)

# Вывод количества созданных узлов и документов
print(f"Created {len(nodes)} nodes from {len(documents)} documents")

In [ ]:
# Вывод текста и метаданных первых трех узлов
for i in range(3):
    print(f"Chunk {i + 1}:")
    print("Text:")
    print(nodes[i].text)
    print("------------------")
    print(f"Title: {nodes[i].metadata['title']}")
    print(f"Link: {nodes[i].metadata['link']}")
    print(f"Date: {nodes[i].metadata['date']}")
    print(f"Author: {nodes[i].metadata['author']}")
    print("------------------")

In [ ]:
# Определение устройства (GPU, если доступен, иначе CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка модели для векторных представлений
embed_model = SentenceTransformer('distiluse-base-multilingual-cased-v2').to(device)

In [ ]:
# Загрузка модели и токенизатора
adapt_model_name = "IlyaGusev/saiga_mistral_7b_lora"
base_model_name = "Open-Orca/Mistral-7B-OpenOrca"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
# Установка токена заполнения
tokenizer.pad_token = tokenizer.eos_token

# Загрузка модели с использованием PEFT
model = AutoPeftModelForCausalLM.from_pretrained(adapt_model_name, device_map={"": device}, torch_dtype=torch.bfloat16).to(device)

# Определение класса Saiga
class Saiga(CustomLLM):
    num_output: int = 512 # Количество выходных токенов
    model_name: str = "Saiga" # Имя модели
    model: Any = None

    def __init__(self, model, num_output):
        super(Saiga, self).__init__()
        self.model = model # Инициализация модели
        self.num_output = num_output # Инициализация количества выходных токенов

    @property
    def metadata(self) -> LLMMetadata:  # Метод для получения метаданных модели
        """Get LLM metadata."""
        return LLMMetadata(
            num_output=self.num_output,
            model_name=self.model_name,
        )

    @llm_completion_callback() # Декоратор для обратного вызова
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse: # Метод для генерации ответа
        inputs = tokenizer(prompt, return_tensors="pt").to(device) # Токенизация входного запроса
        with torch.no_grad(): # Отключение градиентов
            outputs = self.model.generate( # Генерация ответа
                input_ids=inputs["input_ids"],  # Входные идентификаторы
                attention_mask=inputs["attention_mask"], # Маска внимания
                max_new_tokens=self.num_output, # Максимальное количество новых токенов
                temperature=0.3,
                top_p=0.5,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,  # Включение выборки
                **kwargs
            )
        text = tokenizer.decode(outputs[0], skip_special_tokens=True) # Декодирование выходного текста
        return CompletionResponse(text=text)

    @llm_completion_callback()
    def stream_complete(self, prompt: str, **kwargs: Any) -> CompletionResponseGen:
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        response = ""
        for output in self.model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=self.num_output,
            temperature=0.3,
            top_p=0.5,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            **kwargs,
            stream=True
        ):
            token = tokenizer.decode(output, skip_special_tokens=True)
            response += token # Добавление токена к ответу
            yield CompletionResponse(text=response, delta=token)

# Создание экземпляра класса Saiga
saiga = Saiga(model, 512)

In [ ]:
# Функция для получения эмбеддингов
def get_embedding(sentence):
    return embed_model.encode(sentence, convert_to_tensor=True).to(device)

# Вычисление эмбеддингов для всех узлов
embeddings = [get_embedding(node.text) for node in nodes]

In [ ]:
# Удаление индекса "articles", если он существует
es.indices.delete(index="articles", ignore_unavailable=True)

In [ ]:
#es = Elasticsearch(['https://localhost:9200'], basic_auth=('elastic', 'JstXEztaqQEigZ8TOTxl'), ca_certs="/content/elasticsearch-8.8.0/config/certs/http_ca.crt")

# Настройки для индексации
settings = {
    "analysis": {
        "analyzer": {
            "my_custom_index_analyzer": {
                "tokenizer": "standard",
                "filter": ["lowercase"], # Применение фильтра для приведения к нижнему регистру
            },
            "my_custom_search_analyzer": {
                "tokenizer": "standard",
                "filter": ["lowercase", "my_synonym_filter"],
            },
        },
        "filter": {
            "my_synonym_filter": {
                "type": "synonym_graph",
                "synonyms": [ # Синонимы для фильтрации
                    "отдел, департамент",
                    "обращение, запрос",
                    "клиент, заказчик",
                    "контракт, соглашение",
                    "контрагент, заказчик"
                ],
                "updateable": True,
            }
        },
    }
}

# Определение маппинга для индекса
mappings = {
    "properties": {
        "text": {
            "type": "text",
            "analyzer": "my_custom_index_analyzer", # Анализатор для индексации
            "search_analyzer": "my_custom_search_analyzer", # Анализатор для поиска
        },
        "link": {"type": "keyword"}, # Тип поля для ссылки
        "embedding": {"type": "dense_vector", "dims": 512} # Тип поля для векторного представления
    }
}

# Создание индекса "articles"
try:
    es.indices.create(index="articles", mappings=mappings, settings=settings)
    print("Индекс 'articles' создан")
except Exception as e:
    print(f"Ошибка при создании индекса 'articles': {e}")

In [ ]:
# Функция для поиска статей в Elasticsearch с использованием векторных представлений
def search_articles(query, top_k=5):
    query_embedding = get_embedding(query).tolist() # Получение эмбеддинга для запроса
    script_query = {
        "script_score": {
            "query": {"match_all": {}}, # Запрос для поиска всех документов
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0", # Использование косинусного сходства
                "params": {"query_vector": query_embedding} # Вектор запроса
            }
        }
    }
    response = es.search(index="articles", body={ # Выполнение поиска
        "size": top_k, # Количество возвращаемых результатов
        "query": script_query, # Запрос
        "_source": ["text", "link"] # Поля, которые нужно вернут
    })
    return response['hits']['hits'] # Возврат результатов поиска

In [ ]:
# Функция для генерации ответа
def generate_response(question, context):
    prompt = question + context # Объединение вопроса и контекста в один запрос
    response = saiga.complete(prompt) # Генерация ответа с использованием модели Saiga
    return response.text

In [ ]:
# Создаем экземпляр класса LongContextReorder для переупорядочивания контекста
reorderer = LongContextReorder()

# Функция для ответа на вопрос с возможностью переупорядочивания результатов
def answer_question_with_reorder(query, reorder=True, print_results=False):
    results = search_articles(query) # Поиск статей по запросу
    if print_results:
        for hit in results: # Перебор найденных результатов
            print(f"{hit['_source']['text']}\n--------")
    if reorder:
        results = reorderer.transform_documents(results) # Переупорядочивание результатов
    context = " ".join([hit["_source"]["text"] for hit in results])
    response = generate_response(query, context) # Генерация ответа на основе запроса и контекста
    link = results[0]["_source"]["link"] if results else None # Получение ссылки на первую статью, если есть результаты
    return {
        "response": response,
        "link": link
    }

In [ ]:
# Импорт необходимых классов для работы с Elasticsearch
from langchain_community.vectorstores import ElasticsearchStore
from elasticsearch import Elasticsearch

# Создание экземпляра ElasticsearchStore
es_vector_store = ElasticsearchStore(
    index_name="articles",
    embedding=embed_model,
    es_connection=es,
    vector_query_field='vector',  # Поле для векторного запроса
    query_field='text',            # Поле для текстового запроса
    distance_strategy='COSINE'     # Стратегия расстояния для поиска
)

In [ ]:
# Определение метаданных и информации о векторном хранилище
vector_store_info = VectorStoreInfo(
    content_info="Библиотека статей для чат-бота",
    metadata_info=[
        MetadataInfo(
            name="title",
            type="str",
            description="Заголовок статьи"
        ),
        MetadataInfo(
            name="text",
            type="str",
            description="Текст статьи"
        ),
        MetadataInfo(
            name="link",
            type="str",
            description="Ссылка на полную статью"
        ),
        MetadataInfo(
            name="date",
            type="date",
            description="Дата публикации статьи"
        ),
        MetadataInfo(
            name="author",
            type="str",
            description="Автор статьи"
        )
    ],
)

In [ ]:
# Создание извлекателя на основе ElasticsearchStore
retriever = es_vector_store.as_retriever()

In [ ]:
# Настройка переранжирования
rerank = SentenceTransformerRerank(
    top_n = 2, # Количество верхних результатов для переупорядочивания
    model = "BAAI/bge-reranker-base"  # Модель для переупорядочивания
)

In [ ]:
# Определение шаблона промпта для вопросно-ответной системы
qa_prompt_tmpl_str = """\
Контекстная информация о статье:
---------------------
{context_str}
---------------------
Пример вопроса и ответа:
---------------------
Вопрос: {example_question}
Ответ: {example_answer}
---------------------
Текущий запрос пользователя:
---------------------
{query_str}
---------------------
Ответьте на запрос, используя контекстную информацию и пример. Укажите ссылку на источник, если это возможно.
Ответ: \
"""

# Создание шаблона промпта на основе строки
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

# Контекстная информация о статье
context_str = """
Клиент просит отключить уведомления (E-mail; ВК; Viber; СМС; Автообзвон; Обзвон)
Действия при поступлении запроса от клиента
Для отключения уведомлений, необходимо уточнить у клиента и отправить на почту pecom@pecom.ru:
Тема письма: Отказ от уведомлений
Причину отказа от уведомлений (которую назвал клиент)
Номер телефона или E-mail – который отключаем
Канал оповещения (E-mail или ВК или Viber или СМС или Автообзвон или Обзвон)
Наименование и ИНН/ФИО и данные документа контрагента, которому требуется отключить уведомления
Номер телефона, с которого звонил клиент
Обработка запроса на почте pecom@pecom.ru
Занесение контакта в список отказа от уведомлений
Если необходимо отключить уведомления определённому клиенту или группе лиц, по одному или нескольким каналам связи (E-mail; ВК; Viber; СМС; Автообзвон; Обзвон; и т.д.), то в Пегасе, на вкладке «Обработки» необходимо нажать «Отказ от уведомлений» и выполнить ряд действий:
После нажатия откроется окно: «Отказ от уведомлений»
Имеются два способа занесения контакта в «Чёрный список»:
Через поиск по ИНН.
Через «Добавление в отказ от уведомлений».
Поиск по ИНН
В поле «ИНН Контрагента» - ввести номер ИНН клиента и нажать кнопку «Найти контактные данные КА по ИНН»:
В открывшимся окне выбрать «галочками» контакты которые необходимо заблокировать (телефон или E-mail) и нажать кнопку «Добавление в отказ от уведомлений»:
Открывается «Режим добавления в ЧС», (номера телефонов и E-mail блокируются в разных меню).
Выбрать «галочками» необходимый канал блокировки: телефон или E-mail, прописать комментарии в строку «Примечание» и «Причина» и нажать кнопку «Записать элемент»:
Результат: выбранные контакты внесены в отказ от уведомлений и отображаются на вкладке «Активные телефоны ЧС»:
Добавление в отказ от уведомлений
В окне «Отказ от уведомлений» нажать кнопку «Добавление в отказ от уведомлений»:
В открывшемся окне, в строку «Телефон» вбить необходимый контакт, (как ниже в примере), «галочками» отметить необходимый канал отказа от уведомлений, прописать примечание и причину, нажать кнопку «Записать элемент».
Если необходимо заблокировать почтовое уведомление, нужно поставить галочку «E-mail», вбить почтовый адрес, прописать примечание и причину, нажать кнопку «Записать элемент».
Результат: выбранные контакты внесены в ЧС и отображаются на вкладке «Активные телефоны ЧС»:
Исключение номера телефона или E-mail из списка «Отказ от уведомлений»
В окне «Отказ от уведомлений», на вкладке «Активные телефоны ЧС», встать/выделить необходимый контакт для исключения и нажать кнопку «Удаление из отказа от уведомлений»:
В открывшимся окне «Режим удаления из ЧС: Отказ от уведомлений», прописать примечание и причину, нажать кнопку «Записать элемент»:
Результат: контакт пропадает из «Активных телефонов ЧС», появится запись/строка на вкладке «Удалённые телефоны из ЧС».
"""

# Пример вопроса и ответа
example_question = "Как занести контакт в черный список?"
example_answer = "Имеются два способа занесения контакта в «Чёрный список»: Через поиск по ИНН. Через «Добавление в отказ от уведомлений»."


In [ ]:
# Создание синтезатора ответов с использованием модели Saiga и шаблона
response_synthesizer = get_response_synthesizer(
    llm=saiga,
    text_qa_template=qa_prompt_tmpl # Шаблон для вопросно-ответной системы
)

In [ ]:
# Создание расширенной RAG системы
advanced_rag_query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer, # Синтезатор ответов
    node_postprocessors = [rerank], # Постобработчики узлов
)

In [ ]:
# Функция для интерактивного поиска с переупорядочиванием
def interactive_search_with_reorder():
    while True: # Бесконечный цикл для интерактивного ввода
        inp_question = input("Пожалуйста, введите вопрос: ")
        result = answer_question_with_reorder(inp_question, reorder=True, print_results=True)
        print("\nОтвет:") # Вывод заголовка ответа
        print(result["response"]) # Вывод текста ответа
        if result["link"]: # Если есть ссылка на источник
            print(f"\nСсылка на источник: {result['link']}") # Вывод ссылки
        print("\n\n========\n") # Разделитель для удобства чтения

# Основная функция, запускающая интерактивный поиск
if __name__ == "__main__":
    interactive_search_with_reorder()